In [ ]:
import csv
import string
import re
import pandas as pd
from collections import Counter
from datetime import datetime
from src import common

In [ ]:
def to_clean_tokens(firmname):
    """"""
    
    full_punctuation = string.punctuation + '’'
    translator = str.maketrans(full_punctuation, ' '*len(full_punctuation))
    
    decrease = 1
    while decrease > 0:
        start_len = len(firmname)
        firmname = firmname.replace('  ', ' ')
        decrease = start_len - len(firmname)
    tokens = firmname.translate(translator).split()
    tokens_cleans = [re.sub("[^0-9a-zA-Z]+", "", t).lower() for t in tokens]
    tokens_filtered = list(set([t for t in tokens_cleans if len(t) > 0]))#.sort(reverse=True)
    
    return tokens_filtered


def sequence_uniqueness(tokens, token_frequency_dict):
    """"""
    
    return sum(1 / token_frequency_dict[t] ** 0.5 for t in tokens)


def name_similarity(name_a, name_b, token_frequency):
    """"""
    
    a_tokens = set(name_a)
    b_tokens = set(name_b)
    a_uniq = sequence_uniqueness(name_a, token_frequency)
    b_uniq = sequence_uniqueness(name_b, token_frequency)
    if a_uniq == 0 or b_uniq == 0:
        return 0
    else:
        return sequence_uniqueness(a_tokens.intersection(b_tokens), token_frequency) / (a_uniq * b_uniq) ** 0.5

    
def build_token_frequency_table(token_lists):
    """"""
    
    tokens = [str(token) for s in token_lists for token in s]
    return Counter(tokens)

def column_to_date(dataf, column):
    
    raw_dates = dataf[column].values

    new_dates = []
    for rd in raw_dates:
        succeed = False
        try:
            nd = datetime.strptime(rd, '%d/%m/%Y')
            succeed = True
        except:
            pass
        if not succeed:
            try:
                nd = datetime.strptime(f'01/01/{rd}', '%d/%m/%Y')
                succeed = True
            except:
                pass
        if succeed:
            new_dates.append(nd.strftime('%Y-%m-%d'))
        else:
            new_dates.append(None)

    dataf[column] = new_dates
    
    return dataf

def match_firm_hash(dfbase, dfmatch, min_score=0.5):
    """"""

    base_firm_names = list(dfbase['firm'].values)
    base_firm_hashed = list(dfbase['firmhash'].values)

    firm_hash_dict = {}
    for bn, bh in zip(base_firm_names, base_firm_hashed):
        firm_hash_dict[bn] = {
            'hash': bh,
            'tokens': to_clean_tokens(bn)
        }

    match_firm_names = list(dfmatch['firm'].values)
    match_firms_tokenized = [to_clean_tokens(f) for f in match_firm_names]
    match_firm_dict = {}
    for mn, mt in zip(match_firm_names, match_firms_tokenized):
        match_firm_dict[mn] = {
            'tokens': mt,
        }

    all_companies_tokenized = [*[v['tokens'] for v in firm_hash_dict.values()], *match_firms_tokenized]

    token_frequency = build_token_frequency_table(all_companies_tokenized)
    
    no_match = 0
    for mk, mv in match_firm_dict.items():
        match_scores = []
        max_score = 0
        best_match = None
        for bk, bv in firm_hash_dict.items():
            try:
                subscore = name_similarity(bv['tokens'], mv['tokens'], token_frequency)
            except:
                print('Error on similarity match')
                print(bv['tokens'])
                print(mk)
            if subscore > min_score:
                if subscore > max_score:
                    max_score = subscore
                    best_match = bv['hash']
        match_firm_dict[mk]['hash'] = best_match
        match_firm_dict[mk]['match_score'] = max_score
        if max_score <= min_score:
            no_match += 1
            print(f'No match found for: {mk}')

    df_hashmatch = pd.DataFrame({
        'firm': match_firm_dict.keys(),
        'firmhash': [v['hash'] for v in match_firm_dict.values()]
    })
    df_hashmatch['firmhash'] = df_hashmatch['firmhash'].fillna(0).astype(int)

    dfmatch = dfmatch.merge(df_hashmatch, on='firm', how='left')

    return dfmatch.drop(columns=['firm']), no_match, token_frequency

In [ ]:
dffortune = pd.read_csv('../data/fortune/f500_full_firm_data.csv')
dffortune_sample = dffortune.loc[lambda x: (x['ranking']<=300) & (x['include']==True)].drop(columns=['include', 'ranklabel']).reset_index(drop=True)

dffortune_sample_r_growth = dffortune_sample.loc[lambda x: (~x['reputation_score_2020'].isnull()) & (~x['reputation_score_2019'].isnull())][['reputation_score_2020', 'reputation_score_2019']]
dffortune_sample_r_growth = dffortune_sample_r_growth.loc[lambda x: (x['reputation_score_2020']!='-') & (x['reputation_score_2019']!='-')]

dffortune_sample_r_growth = dffortune_sample_r_growth.astype(float).round(5)
dffortune_sample_r_growth['reputation_score_growth'] = (dffortune_sample_r_growth['reputation_score_2020'] - dffortune_sample_r_growth['reputation_score_2019']) / dffortune_sample_r_growth['reputation_score_2019']

dffortune_sample = pd.concat([dffortune_sample, dffortune_sample_r_growth[['reputation_score_growth']]], axis=1)

In [ ]:
df_pp_features = pd.read_csv('../data/policies/features/firm_pp_features_0.2.0.csv')
df_pp_features = df_pp_features.drop(columns=['firm'])

In [ ]:
df_prc_data_breaches = pd.read_csv('../data/breaches/prc_firm_data_breach_matches.csv')

In [ ]:
df_control_1 = pd.read_csv('../data/dbs/date_of_incorporation_and_stock_turnover.csv')

df_control_1['stock_turnover'] = df_control_1['stock_turnover'].str.replace(',', '.')
df_control_1['stock_turnover'] = df_control_1['stock_turnover'].replace('n.a.', None)
df_control_1['stock_turnover'] = df_control_1['stock_turnover'].replace('n.s.', None)
df_control_1['stock_turnover'] = df_control_1['stock_turnover'].astype(float)

df_control_1 = column_to_date(df_control_1, 'date_of_incorporation')

df_control_1['age_in_years'] = round((pd.Timestamp.now() - pd.to_datetime(df_control_1['date_of_incorporation'])).dt.days / 364.24, 1)

df_control_1, _, __ = match_firm_hash(dffortune_sample, df_control_1)
print(f'n no match: {_}')

In [ ]:
df_contron_2 = pd.read_csv('../data/dbs/return_on_assets.csv')
df_contron_2 = df_contron_2[['firm', 'return_on_assets']].copy()
df_contron_2['return_on_assets'] = df_contron_2['return_on_assets'].str.replace(',', '.').astype(float)
df_contron_2, _, __ = match_firm_hash(dffortune_sample, df_contron_2)
print(f'n no match: {_}')

In [ ]:
df_employee = pd.read_csv('../data/dbs/employee_satisfaction_glassdoor.csv')
df_employee = df_employee.drop(columns=['Industry', 'Sector'])
df_employee.columns = ['_'.join(to_clean_tokens(c)) for c in df_employee.columns]
df_employee, _, __ = match_firm_hash(dffortune_sample, df_employee)
print(f'n no match: {_}')

In [ ]:
df_ig_manuals = pd.read_csv('../data/dbs/information_governance_practises_manuals.csv')
df_ig_manuals['privacy_policy_url'] = df_ig_manuals['Privacy Policy URL corrected']
df_ig_manuals['privacy_policy_url'] = df_ig_manuals['privacy_policy_url'].fillna(df_ig_manuals['Privacy Policy ULR'])
df_ig_manuals = df_ig_manuals.drop(columns=['Industry', 'Sector', 'Collector ', 'Comment', 'Privacy Policy ULR', 'Privacy Policy URL corrected'])
df_ig_manuals.columns = ['_'.join(to_clean_tokens(c)) for c in df_ig_manuals.columns]
df_ig_manuals, _, __ = match_firm_hash(dffortune_sample, df_ig_manuals)
print(f'n no match: {_}')

In [ ]:
df_iss_share_pros = pd.read_csv('../data/dbs/iss_shareholder_proposals.csv')
df_iss_share_pros.columns = ['_'.join(to_clean_tokens(c)) for c in df_iss_share_pros.columns]
df_iss_share_pros = column_to_date(df_iss_share_pros, 'date_meeting')
df_iss_share_pros = df_iss_share_pros.rename(columns={'company_name': 'firm'})

df_iss_share_pros_count = df_iss_share_pros.groupby('firm', as_index=False)[['other_status']].count().rename(columns={'other_status': 'number_of_shareholder_proposals'})

df_iss_share_pros_count, _, __ = match_firm_hash(dffortune_sample, df_iss_share_pros_count)
print(f'n no match: {_}')

In [ ]:
df_blended = dffortune_sample \
.merge(df_pp_features, how='left', on='firmhash') \
.merge(df_prc_data_breaches, how='left', on='firmhash') \
.merge(df_control_1, how='left', on='firmhash') \
.merge(df_contron_2, how='left', on='firmhash') \
.merge(df_employee, how='left', on='firmhash') \
.merge(df_iss_share_pros_count, how='left', on='firmhash') \
.merge(df_ig_manuals, how='left', on='firmhash')

In [ ]:
df_blended['legislation_complied_with'] = df_blended['legislation_complied_with'].str.lower()

In [ ]:
df_blended.to_csv('../data/modelinput/information_governance_full_dataset.csv',
                  index=False,
                  quoting=csv.QUOTE_NONNUMERIC,
                  quotechar='"')